In [ ]:
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

from mu_density import *

In [ ]:
config = {
    "linac_logfile_data_directory": "S:\\Physics\\Programming\\data\\LinacLogFiles",
    "machine_types": {
        "elekta-agility": {
            "max_leaf_gap": 400,
            "leaf_pair_widths": [
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5
            ]
        }
    }    
}

grid_resolution = 5/3  # mm
machine_type = 'elekta-agility'

max_leaf_gap = config['machine_types'][machine_type]['max_leaf_gap']
leaf_pair_widths = np.array(config['machine_types'][machine_type]['leaf_pair_widths'])

In [ ]:
test_files = glob(os.path.join(
    config["linac_logfile_data_directory"],
    'indexed', '*', '012194*', 'clinical', '*_VMAT', '*', '*.trf'
))

delivery_data = pull_single_logfile(test_files[0])

In [ ]:
mu = np.array(delivery_data.monitor_units)
mlc = np.swapaxes(delivery_data.mlc, 0, 2)
jaw = np.swapaxes(delivery_data.jaw, 0, 1)

In [ ]:
def determine_leaf_y(leaf_pair_widths):
    total_leaf_widths = np.sum(leaf_pair_widths)
    leaf_y = np.cumsum(leaf_pair_widths) - leaf_pair_widths/2 - total_leaf_widths/2
    
    return leaf_y

In [ ]:
def determine_full_grid(max_leaf_gap, grid_resolution, leaf_pair_widths):
    leaf_x = np.arange(
        -max_leaf_gap/2,
        max_leaf_gap/2 + grid_resolution,
        grid_resolution).astype('float')
    
    leaf_y = determine_leaf_y(leaf_pair_widths)
    
    initial_leaf_grid_y_pos = leaf_y[len(leaf_y)//2]
    
    total_leaf_widths = np.sum(leaf_pair_widths)
    top_grid_pos = (
        (total_leaf_widths/2 - initial_leaf_grid_y_pos) // grid_resolution *
        grid_resolution + initial_leaf_grid_y_pos)

    bot_grid_pos = (
        initial_leaf_grid_y_pos -
        (total_leaf_widths/2 + initial_leaf_grid_y_pos) // grid_resolution *
        grid_resolution)

    grid_y = np.arange(
        bot_grid_pos, top_grid_pos + grid_resolution, grid_resolution)
    
    grid_xx, grid_yy = np.meshgrid(leaf_x, grid_y)
    
    return grid_xx, grid_yy

In [ ]:
def determine_calc_grid_and_adjustments(mlc, jaw, grid_resolution):
    min_x = np.floor(np.min(-mlc[:, :, 0]) / grid_resolution) * grid_resolution
    max_x = np.ceil(np.max(mlc[:, :, 1]) / grid_resolution) * grid_resolution

    min_y = np.min(-jaw[:, 0])
    max_y = np.max(jaw[:, 1])

    leaf_x = np.arange(
        min_x, max_x + grid_resolution, grid_resolution
    ).astype('float')

    leaf_y = determine_leaf_y(leaf_pair_widths)

    leaf_xx, _ = np.meshgrid(leaf_x, leaf_y)

    initial_leaf_grid_y_pos = leaf_y[len(leaf_y)//2]

    top_grid_pos = (
        (max_y - initial_leaf_grid_y_pos) // grid_resolution *
        grid_resolution + initial_leaf_grid_y_pos)

    bot_grid_pos = (
        initial_leaf_grid_y_pos -
        (-min_y + initial_leaf_grid_y_pos) // grid_resolution *
        grid_resolution)

    grid_y = np.arange(
        bot_grid_pos, top_grid_pos + grid_resolution, grid_resolution)

    grid_leaf_map = np.argmin(
        np.abs(grid_y[:, None] - leaf_y[None, :]), axis=1)

    grid_xx, grid_yy = np.meshgrid(leaf_x, grid_y)
    adjusted_grid_leaf_map = grid_leaf_map - np.min(grid_leaf_map)

    leaves_to_be_calced = np.unique(grid_leaf_map)
    adjusted_mlc = mlc[:, leaves_to_be_calced, :]
    adjusted_leaf_xx = leaf_xx[leaves_to_be_calced, :]
    
    return grid_xx, grid_yy, adjusted_grid_leaf_map, adjusted_mlc, adjusted_leaf_xx

In [ ]:
def remove_irrelevant_control_points(mu, mlc, jaw):
    mu_diff = np.diff(mu)
    no_change = mu_diff == 0
    no_change_before = no_change[0:-1]
    no_change_after = no_change[1::]

    no_change_before_and_after = no_change_before & no_change_after
    irrelevant_control_point = np.hstack(
        [no_change[0], no_change_before_and_after, no_change[-1]])
    control_points_to_use = np.invert(irrelevant_control_point)

    mu = mu[control_points_to_use]
    mlc = mlc[control_points_to_use, :, :]
    jaw = jaw[control_points_to_use, :]
    
    return mu, mlc, jaw

In [ ]:
AGILITY_LEAF_PAIR_WIDTHS = [
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
    5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5
]


def calc_mu_density(mu, mlc, jaw, grid_resolution=1, max_leaf_gap=400,
                    leaf_pair_widths=AGILITY_LEAF_PAIR_WIDTHS, ram_fraction=0.8):
    leaf_pair_widths = np.array(leaf_pair_widths)
    min_number_of_sections = 20

    number_of_sections = min_number_of_sections
    current_index = 0

    mu, mlc, jaw = remove_irrelevant_control_points(mu, mlc, jaw)
    
    (
        grid_xx, grid_yy, adjusted_grid_leaf_map, adjusted_mlc, adjusted_leaf_xx
    ) = determine_calc_grid_and_adjustments(mlc, jaw, grid_resolution)

    final_index = len(mu)
    max_index = 0

    mu_density = np.zeros_like(grid_xx)

    while max_index < final_index:
        max_index = calc_max_index(
            current_index, number_of_sections, final_index)

        slice_to_check = slice(current_index, max_index, 1)
        mu_density_of_slice, ram_used = calc_mu_density_over_slice(
            mu, adjusted_mlc, jaw, slice_to_check,
            grid_xx, grid_yy, adjusted_leaf_xx, adjusted_grid_leaf_map)
        mu_density += mu_density_of_slice

        current_index = current_index + number_of_sections

        current_ram_fraction = ram_used / psutil.virtual_memory().available
        if current_ram_fraction != 0:
            number_of_sections = number_of_sections * int(
                np.floor(ram_fraction / current_ram_fraction))

            if number_of_sections < min_number_of_sections:
                number_of_sections = min_number_of_sections
                
    full_grid_xx, full_grid_yy = determine_full_grid(
        max_leaf_gap, grid_resolution, leaf_pair_widths)
    
    xx_from, xx_to = np.where(np.abs(full_grid_xx[None,0,:] - grid_xx[0,:,None]) < 0.0001)
    yy_from, yy_to = np.where(np.abs(full_grid_yy[None,:,0] - grid_yy[:,0,None]) < 0.0001)
    
    full_grid_mu_density = np.zeros_like(full_grid_xx)
    full_grid_mu_density[np.ix_(yy_to, xx_to)] = mu_density[np.ix_(yy_from, xx_from)]

    return full_grid_xx, full_grid_yy, full_grid_mu_density


In [ ]:
grid_xx, grid_yy, mu_density = calc_mu_density(
    mu, mlc, jaw, grid_resolution=grid_resolution)

In [ ]:
plt.pcolormesh(
    grid_xx, 
    grid_yy,
    mu_density)
plt.colorbar()